In [17]:
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
import torch

# MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
processor = AutoProcessor.from_pretrained(MODEL_ID)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID)


# 비전 인코더만 가져오기
vision_encoder = model.model.visual
# vision_encoder = torch.nn.DataParallel(model.model.visual).eval().cuda()

vision_encoder.eval().cuda()


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]


Qwen2_5_VisionTransformerPretrainedModel(
  (patch_embed): Qwen2_5_VisionPatchEmbed(
    (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
  )
  (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
  (blocks): ModuleList(
    (0-31): 32 x Qwen2_5_VLVisionBlock(
      (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
      (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
      (attn): Qwen2_5_VLVisionSdpaAttention(
        (qkv): Linear(in_features=1280, out_features=3840, bias=True)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (mlp): Qwen2_5_VLMLP(
        (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
        (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
        (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
        (act_fn): SiLU()
      )
    )
  )
  (merger): Qwen2_5_VLPatchMerger(
    (ln_q): Qwen2RMSNorm((1280,), eps=1e-06)
    (mlp): Sequential(
      (0): Linear(in

In [18]:
from PIL import Image
import requests
import torch

# 예시 이미지 불러오기
img = Image.open(requests.get("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", stream=True).raw)

# 이미지 전처리 (output: pixel_values, grid_thw)
processed = processor.image_processor(images=img, return_tensors="pt")
pixel_values = processed["pixel_values"].cuda()        # shape: (1, 3, D, H, W)
grid_thw     = processed["image_grid_thw"].cuda()      # shape: (1, 3)


In [19]:
pixel_values.shape

torch.Size([14308, 1176])

In [20]:
grid_thw.shape

torch.Size([1, 3])

In [21]:
with torch.no_grad():
    out = vision_encoder(pixel_values, grid_thw)  # shape: (tokens, dim=2048)


In [22]:
out.shape

torch.Size([3577, 2048])

In [23]:
vision_encoder.config.out_hidden_size


2048

In [24]:
vision_encoder.config

Qwen2_5_VLVisionConfig {
  "depth": 32,
  "fullatt_block_indexes": [
    7,
    15,
    23,
    31
  ],
  "hidden_act": "silu",
  "hidden_size": 1280,
  "in_channels": 3,
  "in_chans": 3,
  "initializer_range": 0.02,
  "intermediate_size": 3420,
  "model_type": "qwen2_5_vl",
  "num_heads": 16,
  "out_hidden_size": 2048,
  "patch_size": 14,
  "spatial_merge_size": 2,
  "spatial_patch_size": 14,
  "temporal_patch_size": 2,
  "tokens_per_second": 2,
  "torch_dtype": "float32",
  "transformers_version": "4.52.3",
  "window_size": 112
}

In [25]:
import sys
import os
sys.path.append(os.path.abspath("../vlm_forgetting"))

In [26]:
from attentive_pooler import AttentivePooler, AttentiveClassifier

In [27]:
attentive_pooler = AttentiveClassifier(
        embed_dim=vision_encoder.config.out_hidden_size,
        num_heads=vision_encoder.config.num_heads,
        depth=1,
        num_classes=10,

    ).cuda()
out = out.cuda()
out.shape

torch.Size([3577, 2048])

In [28]:
attentive_pooler(out)

ValueError: not enough values to unpack (expected 3, got 2)